In [55]:
import pickle
import numpy as np
import biom
import csv

In [2]:
with open ("oral_medium.p", "rb") as f:
    d = pickle.load(f)

In [3]:
d.keys()

dict_keys(['theta', 'Ytrain', 'Ytest', 'Vtrain', 'Vtest'])

In [4]:
d['theta'].shape

(20, 21)

In [5]:
d['theta'].dtype

dtype('float64')

In [130]:
len(d['Ytrain'])

22

In [131]:
len(d['Ytest'])

6

# top 20 taxa + the rest


Note: we set the rest of taxa (i.e. top 21, 22, ....)  to be taxa0

In [6]:
theta = np.zeros((20, 21))

In [7]:
theta[0,0] = 1
theta[0, 1:] = -1

In [8]:
theta[1,1:4] = 1
theta[1, 4:] = -1

In [9]:
theta[2,1] = 1
theta[2,2:4] = -1

In [10]:
theta[3, 4] = 1
theta[3, 5:] = -1

In [11]:
theta[4, 2] = 1
theta[4, 3] = -1

In [12]:
theta[5, 5] = 1
theta[5, 6:9] = -1

In [13]:
theta[6, 4:9] = 1
theta[6, 9:] = -1

In [14]:
theta[7, 6] = 1
theta[7, 7:9] = -1

In [15]:
theta[8, 9] = 1
theta[8, 10:] = -1

In [16]:
theta[9, 7] = 1
theta[9, 8] = -1

In [17]:
theta[10, 10:13] = 1
theta[10, 13:] = -1

In [18]:
theta[11, 10] = 1
theta[11, 11:13] = -1

In [19]:
theta[12, 13:15] = 1
theta[12, 15:] = -1

In [20]:
theta[13, 11] = 1
theta[13, 12] = -1

In [21]:
theta[14, 13] = 1
theta[14, 13] = -1

In [22]:
theta[15, 15] = 1
theta[15, 16:] = -1

In [23]:
theta[16, 16:18] = 1
theta[16, 18:] = -1

In [24]:
theta[17, 16] = 1
theta[17, 17] = -1

In [25]:
theta[18, 18] = 1
theta[18, 19:] = -1

In [26]:
theta[19, 19] = 1
theta[19, 20] = -1

In [27]:
num_taxa = 20

In [28]:
table = biom.load_table("all.biom")

In [29]:
data = table._data.toarray().T

In [30]:
data.shape

(1050, 6400)

In [35]:
sample_ids = table.ids()
taxon_ids = table.ids(axis="observation")

In [100]:
data_norm = data / data.sum(axis=-1, keepdims=True)
data_sum = np.sum(data_norm, axis=0) / np.sum(data_norm)
sel_idxes = [idx for i, idx in enumerate(np.argsort(-data_sum)) if i < num_taxa]
sel_taxon_ids = taxon_ids[sel_idxes]

# Change the order of selected idxes to match the ordering in phylogenetic tree

In [102]:
print("original odering:", sel_idxes)

original odering: [17, 15, 19, 62, 11, 106, 54, 117, 1, 9, 14, 23, 154, 24, 12, 5, 135, 141, 3, 124]


In [103]:
sel_idxes = [154, 54, 124, 3, 135,19, 15, 23, 62, 12, 117, 5, 14, 24, 106, 11, 141,1, 17, 9]

In [104]:
len(sel_idxes)

20

## note: the first taxa, i.e. taxa 0, is the rest of the taxa

In [105]:
trajs, inputs = [], []
traj, Input = None, None
others = []
foods = []
current_host_id = None

with open("map.onlymedtime.txt") as tsv:
    tsv_reader = csv.DictReader(tsv, delimiter="\t")
    for i, line in enumerate(tsv_reader):
        sampled_id = line["#SampleID"]
        host_id = line["hostid"]
        conttime = int(line["conttime"])

        drink = line["drink"]
        eat = line["eat"]
        sleep = line["sleep"]
        other = line["other"]

        if sampled_id not in sample_ids:
            print("host {}'s sample {} at time {} doesn't exist".format(host_id, sampled_id, line["dayhourmin"]))
        else:
            if host_id != current_host_id:
                if traj is not None:
                    trajs.append(np.array(traj))
                    inputs.append(np.array(Input))
                traj, Input = [], []
                traj_start = conttime // 30

            time_idx = float(conttime // 30 - traj_start)

            # observations (counts)
            row_idx = np.where(sample_ids == sampled_id)[0][0]
            sel_taxon_counts = data[row_idx][sel_idxes]                    # (num_taxa,)
            remain_counts = data[row_idx].sum() - sel_taxon_counts.sum()   # scalar
            y = np.concatenate([[time_idx], [remain_counts], sel_taxon_counts])  # NOTE: first remain_counts, then selected counts
            traj.append(y)

            # inputs [drink, eat, sleep] all 0/1
            inputs.append([time_idx, drink != "", eat != "", sleep != ""])

            if other != "":
                others.append(other)
            if eat != "":
                foods.append(eat)


host MA's sample 19231 at time 4:09:40 doesn't exist
host MA's sample 19244 at time 4:16:40 doesn't exist
host MA's sample 19274 at time 5:11:45 doesn't exist
host ME's sample 19813 at time 1:15:00 doesn't exist
host ME's sample 19884 at time 3:15:02 doesn't exist
host MO's sample 19368 at time 1:19:40 doesn't exist
host MO's sample 19395 at time 2:17:35 doesn't exist
host MO's sample 19520 at time 5:21:05 doesn't exist
host MS's sample 19657 at time 3:12:00 doesn't exist
host MS's sample 19688 at time 4:10:07 doesn't exist
host MS's sample 19696 at time 4:13:07 doesn't exist
host MS's sample 19715 at time 4:23:30 doesn't exist


In [106]:
len(inputs)

1707

In [107]:
len(trajs)

853

# TOP 15 taxa

In [114]:
theta = np.zeros((15, 16))

In [115]:
theta[0, 0] = 1
theta[0, 1:] = -1
theta[1, 1:3] = 1
theta[1, 3:] = -1
theta[2, 1] = 1
theta[2, 2] = -1
theta[3, 3:5] = 1
theta[3, 5:] = -1
theta[4, 3] = 1
theta[4, 4:6] = -1
theta[5, 6] = 1
theta[5, 7:] = -1
theta[6, 4] = 1
theta[6, 5] = -1
theta[7, 7:9] = 1
theta[8, 9:] = -1
theta[8, 7] = 1
theta[8, 8] = -1
theta[9, 9:11] = 1
theta[9, 11:] = -1
theta[10, 9] = 1
theta[10, 10] = -1
theta[11, 11] = 1
theta[11, 12:] = -1
theta[12, 12] = 1
theta[12, 13:] = -1
theta[13, 13] = 1
theta[13, 14:] = -1
theta[14, 14] = 1
theta[14, 15] = -1

In [116]:
num_taxa = 15

In [117]:
data_norm = data / data.sum(axis=-1, keepdims=True)
data_sum = np.sum(data_norm, axis=0) / np.sum(data_norm)
sel_idxes = [idx for i, idx in enumerate(np.argsort(-data_sum)) if i < num_taxa]
sel_taxon_ids = taxon_ids[sel_idxes]

# Change the order of selected idxes to match the ordering in phylogenetic tree

In [118]:
print("original odering:", sel_idxes)

original odering: [17, 15, 19, 62, 11, 106, 54, 117, 1, 9, 14, 23, 154, 24, 12]


In [119]:
len(sel_idxes)

15

In [120]:
sel_idxes = [54, 154, 19, 15, 23, 62, 117, 12, 14, 24, 106, 11, 1, 17, 9]

In [121]:
len(sel_idxes)

15

# TOP 10 taxa

In [112]:
theta = np.zeros((10, 11))

In [113]:
theta[0, 0] = 1
theta[0, 1:] = -1
theta[1, 1] = 1
theta[1, 2:] = -1
theta[2, 2:4] = 1
theta[2, 4:] = -1
theta[3, 2] = 1
theta[3, 3] = -1
theta[4, 4] = 1
theta[4, 5:] = -1
theta[5, 5] = 1
theta[5, 6:] = -1
theta[6, 6:8] = 1
theta[6, 8:] = -1
theta[7, 6] = 1
theta[7, 7] = -1
theta[8, 8] = 1
theta[8, 9:] = -1
theta[9, 9] = 1
theta[9, 10] = -1

In [124]:
num_taxa = 10

In [125]:
data_norm = data / data.sum(axis=-1, keepdims=True)
data_sum = np.sum(data_norm, axis=0) / np.sum(data_norm)
sel_idxes = [idx for i, idx in enumerate(np.argsort(-data_sum)) if i < num_taxa]
sel_taxon_ids = taxon_ids[sel_idxes]

# Change the order of selected idxes to match the ordering in phylogenetic tree

In [126]:
print("original odering:", sel_idxes)

original odering: [17, 15, 19, 62, 11, 106, 54, 117, 1, 9]


In [127]:
len(sel_idxes)

10

In [128]:
sel_idxes = [54, 15, 19, 117, 62, 11, 106, 9, 17, 1]

In [129]:
len(sel_idxes)

10